### 5. Short-list promising models
We expect you to do some additional research and train at **least one model per team member**.

1. Train mainly quick and dirty models from different categories (e.g. linear, SVM, Random Forests etc) using default parameters
2. Measure and compare their performance
3. Analyse the most significant variables for each algorithm
4. Analyse the types of errors the models make
5. Have a quick round of feature selection and engineering if necessary
6. Have one or two more quick iterations of the five previous steps
7. Short-list the top three to five most promising models, preferring models that make different types of errors

In [1]:
# installing dependencies
!pip3 install pandas

Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/Applications/Xcode.app/Contents/Developer/usr/bin/python3 -m pip install --upgrade pip' command.


In [4]:
# importing the data as dataframes
import pandas as pd

emotions_df_train_ready = pd.read_csv('../exports/merged_emotions_scrambled_train_ready.csv')

pd.set_option('display.max_columns', None)

##### just to not forget the mapping

- 0: 'sadness'
- 1: 'joy'
- 2: 'love'
- 3: 'anger'
- 4: 'fear'
- 5: 'surprise'
- 6: 'neutral'

In [5]:
emotions_df_train_ready.value_counts()

Unnamed: 0  text                                                                                                                                                                                                            label
15          [NAME] is always the dirtiest psychopath                                                                                                                                                                        3        1
400306      i have found the more i suppress the vampire inside me the more human emotions i feel the days i don t feel like feeling anymore are the days i m dangerous                                                     3        1
400224      i describe his gentle nature and how being in his presence made me feel more tranquil                                                                                                                           1        1
400225      Man, what a very tolerant person he is.                              